In [13]:
import autokeras as ak
import numpy as np
import pandas as pd
import tensorflow as tf
import sys

In [14]:
%load_ext autoreload
%autoreload 2

sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:

%store

Stored variables and their in-db values:
X_test_estimated_a_corr             ->                      absolute_humidity_2m:gm3  air
X_test_estimated_b_corr             ->                      absolute_humidity_2m:gm3  air
X_test_estimated_c_corr             ->                      absolute_humidity_2m:gm3  air
const_idx                           -> ([34082, 34083], [10967, 10968, 10969, 10970, 1097
data_A                              ->        pv_measurement       date_forecast  absolut
data_B                              ->        pv_measurement       date_forecast  absolut
data_C                              ->        pv_measurement       date_forecast  absolut
dm                                  -> '<unavailable>'
dml                                 -> '<unavailable>'


In [16]:
# Initialize the structured data regressor.
reg = ak.StructuredDataRegressor(
    overwrite=True, max_trials=3
)  # It tries 3 different models.
# Feed the structured data regressor with training data.
reg.fit(
    # The path to the train.csv file.
    train_file_path,
    # The name of the label column.
    "Price",
    epochs=10,
)
# Predict with the best model.
predicted_y = reg.predict(test_file_path)
# Evaluate the best model with testing data.
print(reg.evaluate(test_file_path, "Price"))

NameError: name 'train_file_path' is not defined